**Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto**

**<h2>Question 1</h2>**

Install and import libraries

In [ ]:
jupyter trust SegmentingAndClusteringNeighborhoodsInToronto.ipynb

SyntaxError: ignored

In [ ]:
pip install geocoder

     |████████████████████████████████| 102kB 6.6MB/s 


In [ ]:
#Import relevant libraries
import pandas as pd
import geocoder
import requests # library to handle requests
import numpy as np
from sklearn.cluster import KMeans # import k-means from clustering stage
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm
import matplotlib.colors as colors

Read the Wikipedia html

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(url)

Take the first table from the html as our dataframe

In [ ]:
df_canada = table[0]
df_canada.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Rename the columns

In [ ]:
df_canada.columns = ['PostalCode','Borough','Neighbourhood']
df_canada.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Drop rows which have their Borough as 'Not assigned'

In [ ]:
df_canada = df_canada[df_canada['Borough']!='Not assigned']

Check if any Neighbourhoods are 'Not assigned', there are none so we can leave the data frame as it is

In [ ]:
df_canada[df_canada['Neighbourhood']=='Not assigned']

,PostalCode,Borough,Neighbourhood


Print out the shape and the number of boroughs and neighbourhoods

In [ ]:
print(df_canada.shape)
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(df_canada['Borough'].unique()),
        df_canada.shape[0]
    )
)

(103, 3)
The dataframe has 10 boroughs and 103 neighbourhoods.


**<h2>Question 2</h2>**

Create new DataFrame with latitudes and longitudes of the postcodes.

In [ ]:
#Create empty list to be populated with dictionaries
coords = []

#Populate the list using geocoder
for postcode in df_canada['PostalCode']:
  g=geocoder.arcgis('{}, Toronto, Ontario'.format(postcode))
  lat_lng_coords = g.latlng
  latitude = lat_lng_coords[0]
  longitude = lat_lng_coords[1]
  coord = {'Postcode': postcode, 'Latitude': latitude, 'Longitude': longitude}
  coords.append(coord)

#Convert list to dataframe
df_coords = pd.DataFrame(coords)

The data frame looks like this:

In [ ]:
df_coords.head()

,Postcode,Latitude,Longitude
0,M3A,43.75245,-79.32991
1,M4A,43.73057,-79.31306
2,M5A,43.65512,-79.36264
3,M6A,43.72327,-79.45042
4,M7A,43.66253,-79.39188


Join this data frame with the original data frame.

In [ ]:
df_canada_ll = df_canada.join(df_coords.set_index('Postcode'), on='PostalCode')
df_canada_ll.reset_index(drop=True, inplace=True)
df_canada_ll.shape

(103, 5)

This is what the data frame looks like:

In [ ]:
df_canada_ll.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66263,-79.52831
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
7,M3B,North York,Don Mills,43.74923,-79.36186
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


**<h2>Question 3</h2>**

<h3>In this section we will explore and cluster the neighbourhoods in Toronto</h3>

Define function to use the explore Foursquare API, we will set the radius to 800m

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Define Foursquare details

In [ ]:
CLIENT_ID = 'ER03M3JR3O2RMSFKDP3XGC40N4UDYHK4XQHKQEE2EQ1ZX4HM' 
CLIENT_SECRET = 'MWHY2HX1OZ3Y2N5JPRMOGJ0JG5SLLNTKUAOAL2RFVJCXAFJA' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

We will only analyse boroughs with 'Toronto' in them, so let's drop the rows which this does not apply to

In [ ]:
df_toronto = df_canada_ll[df_canada_ll['Borough'].str.contains('Toronto', regex=False)]
df_toronto.reset_index(inplace=True)

In [ ]:
print('There are {} unique postal codes.'.format(len(df_toronto['PostalCode'].unique())))
print('There are {} unique neighbourhoods.'.format(len(df_toronto['Neighbourhood'].unique())))

There are 39 unique postal codes.
There are 39 unique neighbourhoods.


Apply the function to the Toronto neighbourhoods to get the nearby venues

In [ ]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [ ]:
toronto_venues.shape

(2904, 7)

In [ ]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 285 unique categories.


In [ ]:
print('There are {} unique neighbourhoods.'.format(len(toronto_venues['Neighbourhood'].unique())))

There are 39 unique neighbourhoods.


This is what the data frame looks like:

In [ ]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,Rooster Coffee,43.651900,-79.365609,Coffee Shop
4,"Regent Park, Harbourfront",43.65512,-79.36264,Sumach Espresso,43.658135,-79.359515,Coffee Shop


<h3>Now we analyse each neighbourhood</h3>

Apply one hot encoding to be able to analyse the neighbourhoods

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,...,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tram Station,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
toronto_onehot.shape

(2904, 286)

Group rows by neighborhood and take the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,...,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tram Station,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.00,0.0,0.0,0.00,0.0,0.01,0.00,0.0,0.01,0.00,0.00,0.00,0.00,0.0,0.01,0.0,0.01,0.03,0.00,0.00,0.0,0.00,0.01,0.01,0.00,0.0,0.03,0.0,0.0,0.0,0.01,0.00,0.0,0.00,0.00,0.01,0.00,0.00,0.00,...,0.00,0.00,0.01,0.0,0.0,0.01,0.00,0.00,0.00,0.00,0.01,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.01,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.01
1,"Brockton, Parkdale Village, Exhibition Place",0.01,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.02,0.00,0.02,0.00,0.01,0.0,0.00,0.0,0.00,0.04,0.01,0.06,0.0,0.00,0.00,0.00,0.01,0.0,0.01,0.0,0.0,0.0,0.00,0.00,0.0,0.01,0.00,0.01,0.01,0.00,0.01,...,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.02,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.01,0.0,0.0,0.02,0.01,0.00,0.00,0.00,0.00,0.0,0.00,0.02,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00
2,"Business reply mail Processing Centre, South C...",0.00,0.0,0.0,0.02,0.0,0.00,0.01,0.0,0.01,0.00,0.02,0.01,0.00,0.0,0.00,0.0,0.00,0.01,0.00,0.02,0.0,0.00,0.00,0.00,0.00,0.0,0.03,0.0,0.0,0.0,0.00,0.02,0.0,0.00,0.01,0.00,0.01,0.00,0.00,...,0.00,0.01,0.00,0.0,0.0,0.01,0.00,0.00,0.01,0.01,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.01,0.03,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.01,0.0,0.01,0.01,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.01
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.0,0.0,0.01,0.0,0.00,0.00,0.0,0.00,0.00,0.01,0.02,0.00,0.0,0.00,0.0,0.00,0.02,0.00,0.02,0.0,0.01,0.00,0.00,0.00,0.0,0.02,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.02,0.00,0.01,...,0.01,0.00,0.00,0.0,0.0,0.01,0.01,0.00,0.01,0.01,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.01,0.00,0.0,0.0,0.00,0.00,0.01,0.01,0.01,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.01,0.0,0.00,0.02
4,Central Bay Street,0.00,0.0,0.0,0.01,0.0,0.00,0.00,0.0,0.02,0.01,0.01,0.01,0.00,0.0,0.00,0.0,0.00,0.01,0.01,0.01,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.02,0.0,0.00,0.00,0.02,0.00,0.01,0.00,...,0.00,0.00,0.00,0.0,0.0,0.01,0.00,0.01,0.02,0.00,0.00,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.01,0.02,0.0,0.0,0.00,0.00,0.01,0.00,0.00,0.00,0.0,0.01,0.02,0.0,0.0,0.0,0.0,0.01,0.00,0.0,0.01,0.01


In [ ]:
toronto_grouped.shape

(39, 286)

Print neighbourhoods with top 5 venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.10
1                Hotel  0.05
2           Restaurant  0.05
3                 Café  0.05
4  Japanese Restaurant  0.05


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.09
1  Coffee Shop  0.08
2          Bar  0.06
3   Restaurant  0.05
4       Bakery  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.06
2        Hotel  0.05
3     Beer Bar  0.03
4      Theater  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.10
1                 Gym  0.05
2                Café  0.05
3  Italian Restaurant  0.04
4                Park  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.08
1                Café  0.04
2     

Write a function to sort the venues in descending order


In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new dataframe and display the top 10 venues for each neighborhood

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Restaurant,Japanese Restaurant,Café,Park,Bakery,Beer Bar,Cocktail Bar,Italian Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bar,Restaurant,Furniture / Home Store,Bakery,Gift Shop,Thrift / Vintage Store,Italian Restaurant,Arts & Crafts Store
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Beer Bar,Theater,Pub,Pizza Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Gym,Italian Restaurant,Park,Restaurant,Japanese Restaurant,Seafood Restaurant,Spa,Diner
4,Central Bay Street,Coffee Shop,Café,Ramen Restaurant,Clothing Store,Seafood Restaurant,Plaza,Park,Japanese Restaurant,Italian Restaurant,Sushi Restaurant


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [ ]:
# set number of clusters
kclusters = 5

# drop the neighbourhood column
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 2, 2, 2, 0, 2, 2, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood


In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge df_toronto with neighborhoods_venues_sorted to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,index,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,2,Coffee Shop,Theater,Park,Bakery,Café,Breakfast Spot,Pub,Restaurant,Performing Arts Venue,Italian Restaurant
1,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,2,Coffee Shop,Café,Sandwich Place,Clothing Store,Boutique,French Restaurant,Bookstore,Park,Smoothie Shop,Bubble Tea Shop
2,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,2,Coffee Shop,Japanese Restaurant,Gastropub,Hotel,Diner,Theater,Ramen Restaurant,Plaza,Pizza Place,New American Restaurant
3,15,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,2,Coffee Shop,Café,Clothing Store,Restaurant,Seafood Restaurant,Hotel,Gastropub,Italian Restaurant,American Restaurant,Cosmetics Shop
4,19,M4E,East Toronto,The Beaches,43.67709,-79.29547,0,Pub,Sandwich Place,Bar,Pizza Place,Coffee Shop,Japanese Restaurant,Furniture / Home Store,Neighborhood,Tea Room,Mexican Restaurant


Plot a map of the resulting clusters

In [ ]:
# Get the coordinates of Toronto

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

There are two main clusters with the most neighbourhoods, cluster 0 and cluster 2. Cluster 2 is concentrated in downtown Toronto and cluster 0 surrounds it. From the categories below, we can see that cluster 0 contains neighbourhoods with many different restaurants and cafes as well as banks and parks. Cluster 2 neighbourhoods have lots of cafes, hotels and shops, as well as restaurants. This makes sense considering the central location of cluster 2 neighbourhoods, catering for workers and tourists, compared with the residential neighbourhoods in cluster 0.

<a id='item5'></a>


These are what the categories look like

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4E,-79.29547,0,Pub,Sandwich Place,Bar,Pizza Place,Coffee Shop,Japanese Restaurant,Furniture / Home Store,Neighborhood,Tea Room,Mexican Restaurant
7,M6G,-79.42071,0,Korean Restaurant,Grocery Store,Coffee Shop,Café,Mexican Restaurant,Park,Pub,Sandwich Place,Ice Cream Shop,Cocktail Bar
9,M6H,-79.43891,0,Coffee Shop,Bar,Bakery,Café,Park,Mexican Restaurant,Sandwich Place,Pharmacy,Middle Eastern Restaurant,Cocktail Bar
11,M6J,-79.41774,0,Bar,Café,Italian Restaurant,Cocktail Bar,Bakery,Vegetarian / Vegan Restaurant,Restaurant,Pizza Place,Coffee Shop,Asian Restaurant
12,M4K,-79.35512,0,Greek Restaurant,Café,Ice Cream Shop,Restaurant,Italian Restaurant,Flower Shop,Bank,Bakery,Spa,Bookstore
14,M6K,-79.42676,0,Café,Coffee Shop,Bar,Restaurant,Furniture / Home Store,Bakery,Gift Shop,Thrift / Vintage Store,Italian Restaurant,Arts & Crafts Store
15,M4L,-79.31467,0,Indian Restaurant,Park,Coffee Shop,Brewery,Gym,Grocery Store,Beach,Restaurant,Sandwich Place,Italian Restaurant
17,M4M,-79.33497,0,Coffee Shop,Pizza Place,Bar,American Restaurant,Diner,Grocery Store,Brewery,Italian Restaurant,Pet Store,Bank
20,M4P,-79.38851,0,Pizza Place,Park,Coffee Shop,Dog Run,Gym,Hotel,Costume Shop,Beer Store,Taco Place,Sushi Restaurant
21,M5P,-79.41440,0,Park,Café,Bank,Middle Eastern Restaurant,Burger Joint,Japanese Restaurant,Bagel Shop,Italian Restaurant,Coffee Shop,Juice Bar


In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M5N,-79.41848,1,Playground,Bank,Café,Garden,Pharmacy,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School


In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,-79.36264,2,Coffee Shop,Theater,Park,Bakery,Café,Breakfast Spot,Pub,Restaurant,Performing Arts Venue,Italian Restaurant
1,M7A,-79.39188,2,Coffee Shop,Café,Sandwich Place,Clothing Store,Boutique,French Restaurant,Bookstore,Park,Smoothie Shop,Bubble Tea Shop
2,M5B,-79.37804,2,Coffee Shop,Japanese Restaurant,Gastropub,Hotel,Diner,Theater,Ramen Restaurant,Plaza,Pizza Place,New American Restaurant
3,M5C,-79.37587,2,Coffee Shop,Café,Clothing Store,Restaurant,Seafood Restaurant,Hotel,Gastropub,Italian Restaurant,American Restaurant,Cosmetics Shop
5,M5E,-79.37306,2,Coffee Shop,Hotel,Restaurant,Japanese Restaurant,Café,Park,Bakery,Beer Bar,Cocktail Bar,Italian Restaurant
6,M5G,-79.38493,2,Coffee Shop,Café,Ramen Restaurant,Clothing Store,Seafood Restaurant,Plaza,Park,Japanese Restaurant,Italian Restaurant,Sushi Restaurant
8,M5H,-79.38258,2,Coffee Shop,Hotel,Café,Theater,Restaurant,Japanese Restaurant,Gastropub,Gym,Vegetarian / Vegan Restaurant,Cosmetics Shop
10,M5J,-79.38076,2,Coffee Shop,Café,Hotel,Restaurant,Park,Theater,Japanese Restaurant,Brewery,Deli / Bodega,Gastropub
13,M5K,-79.38153,2,Hotel,Coffee Shop,Restaurant,Café,Seafood Restaurant,Concert Hall,Italian Restaurant,Japanese Restaurant,Monument / Landmark,Beer Bar
16,M5L,-79.37914,2,Coffee Shop,Hotel,Café,Restaurant,Gastropub,Japanese Restaurant,Beer Bar,Gym,American Restaurant,Concert Hall


In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,M4W,-79.37829,3,Park,Playground,Candy Store,Trail,Grocery Store,Yoga Studio,Electronics Store,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,M4T,-79.38318,4,Park,Grocery Store,Thai Restaurant,Seafood Restaurant,Japanese Restaurant,Candy Store,Rental Car Location,Trail,Playground,Gym
